In [1]:
print("OK")

OK


In [2]:
import os
from dotenv import load_dotenv

In [3]:
#with open('api_key.txt', 'r') as f:
 # api_key = f.read().strip()

In [4]:
from langchain.document_loaders import PyPDFLoader

In [5]:
%pwd

'c:\\Users\\u013709\\How to run LLM'

In [6]:
file_path = "SDG.pdf"
loader = PyPDFLoader(file_path)
data = loader.load()

In [7]:
data=data[1:3]

In [8]:
len(data)

2

In [9]:
question_gen = ""

for page in data:
    question_gen += page.page_content

In [10]:
type(question_gen)

str

In [11]:
question_gen

'IN THE YEAR 2015, LEADERS FROM 193 COUNTRIES OF THE WORLD \nCAME TOGETHER TO FACE THE FUTURE.\nAnd what they saw was daunting. Famines. Drought. Wars. Plagues. Poverty. \nNot just in some faraway place, but in their own cities and towns and villages.\nThey knew things didn’t have to be this way. They knew we had enough  \nfood to feed the world, but that it wasn’t getting shared. They knew there \nwere medicines for HIV and other diseases, but they cost a lot. They knew  \nthat earthquakes and floods were inevitable, but that the high death  \ntolls were not. \nThey also knew that billions of people worldwide shared their hope for a \nbetter future.\nSo leaders from these countries created a plan called the Sustainable \nDevelopment Goals (SDGs). This set of 17 goals imagines a future just 15 years \noff that would be rid of poverty and hunger, and safe from the worst effects of \nclimate change. It’s an ambitious plan. \nBut there’s ample evidence that we can succeed. In the past 15 

In [12]:
question_gen=question_gen[:350]

In [13]:
from langchain.text_splitter import TokenTextSplitter

In [14]:
splitter_ques_gen = TokenTextSplitter(
    model_name= "gpt-3.5-turbo",
    chunk_size= 400,
    chunk_overlap = 25
)

In [15]:
chunk_ques_gen = splitter_ques_gen.split_text(question_gen)

In [16]:
len(chunk_ques_gen)

1

In [17]:
chunk_ques_gen

['IN THE YEAR 2015, LEADERS FROM 193 COUNTRIES OF THE WORLD \nCAME TOGETHER TO FACE THE FUTURE.\nAnd what they saw was daunting. Famines. Drought. Wars. Plagues. Poverty. \nNot just in some faraway place, but in their own cities and towns and villages.\nThey knew things didn’t have to be this way. They knew we had enough  \nfood to feed the world, but tha']

In [18]:
from langchain.docstore.document import Document

In [19]:
document_ques_gen = [Document(page_content = t) for t in chunk_ques_gen]

In [20]:
document_ques_gen

[Document(page_content='IN THE YEAR 2015, LEADERS FROM 193 COUNTRIES OF THE WORLD \nCAME TOGETHER TO FACE THE FUTURE.\nAnd what they saw was daunting. Famines. Drought. Wars. Plagues. Poverty. \nNot just in some faraway place, but in their own cities and towns and villages.\nThey knew things didn’t have to be this way. They knew we had enough  \nfood to feed the world, but tha', metadata={})]

In [21]:
splitter_ans_gen = TokenTextSplitter(
    model_name = 'gpt-3.5-turbo',
    chunk_size = 350,
    chunk_overlap = 25
)

In [22]:
document_answer_gen = splitter_ans_gen.split_documents(
    document_ques_gen
)

In [23]:
document_answer_gen

[Document(page_content='IN THE YEAR 2015, LEADERS FROM 193 COUNTRIES OF THE WORLD \nCAME TOGETHER TO FACE THE FUTURE.\nAnd what they saw was daunting. Famines. Drought. Wars. Plagues. Poverty. \nNot just in some faraway place, but in their own cities and towns and villages.\nThey knew things didn’t have to be this way. They knew we had enough  \nfood to feed the world, but tha', metadata={})]

In [24]:
len(document_answer_gen)

1

In [25]:
from langchain.chat_models import ChatOpenAI

In [26]:
from langchain.llms import CTransformers
llm_ques_gen_pipeline = CTransformers(
    model="llama-2-7b-chat.ggmlv3.q4_0.bin",
    model_type="llama",
    config={
        'max_new_tokens': 128,
        'temperature': 0.01
    }
)

In [27]:
prompt_template = """
You are an expert at creating questions based on coding materials and documentation.
Your goal is to prepare a coder or programmer for their exam and coding tests.
You do this by asking questions about the text below:

------------
{text}
------------

Create questions that will prepare the coders or programmers for their tests.
Make sure not to lose any important information.

QUESTIONS:
"""

In [28]:
from langchain.prompts import PromptTemplate

In [29]:
PROMPT_QUESTIONS = PromptTemplate(template=prompt_template, input_variables=['text'])

In [30]:
refine_template = ("""
You are an expert at creating practice questions based on coding material and documentation.
Your goal is to help a coder or programmer prepare for a coding test.
We have received some practice questions to a certain extent: {existing_answer}.
We have the option to refine the existing questions or add new ones.
(only if necessary) with some more context below.
------------
{text}
------------

Given the new context, refine the original questions in English.
If the context is not helpful, please provide the original questions.
QUESTIONS:
"""
)


In [31]:
REFINE_PROMPT_QUESTIONS = PromptTemplate(
    input_variables=["existing_answer", "text"],
    template=refine_template,
)

In [32]:
from langchain.chains.summarize import load_summarize_chain

In [33]:
ques_gen_chain = load_summarize_chain(llm = llm_ques_gen_pipeline, 
                                          chain_type = "refine", 
                                          verbose = True, 
                                          question_prompt=PROMPT_QUESTIONS, 
                                          refine_prompt=REFINE_PROMPT_QUESTIONS)

In [34]:
def make_llama_request(document_ques_receive):
    try:
        # Assuming document_ques_gen contains the input document
        response = ques_gen_chain.run(document_ques_receive)
        return response
    except Exception as e:
        print(f"Error: {e}")
        time.sleep(5)  # Wait for a few seconds before retrying
        return "None"

# Call the function to make the LLAMA 2 API request
response = make_llama_request(document_ques_gen)

print(response)



> Entering new  chain...


> Entering new  chain...
Prompt after formatting:

You are an expert at creating questions based on coding materials and documentation.
Your goal is to prepare a coder or programmer for their exam and coding tests.
You do this by asking questions about the text below:

------------
IN THE YEAR 2015, LEADERS FROM 193 COUNTRIES OF THE WORLD 
CAME TOGETHER TO FACE THE FUTURE.
And what they saw was daunting. Famines. Drought. Wars. Plagues. Poverty. 
Not just in some faraway place, but in their own cities and towns and villages.
They knew things didn’t have to be this way. They knew we had enough  
food to feed the world, but tha
------------

Create questions that will prepare the coders or programmers for their tests.
Make sure not to lose any important information.

QUESTIONS:


> Finished chain.

> Finished chain.

1. What year did leaders from 193 countries come together to face the future?
2. What did they see when they looked at their own cities and town

In [35]:
#!curl https://api.openai.com/v1/models -H "Authorization: Bearer api_key"

In [10]:
#%pip install llama-cpp-python
#%pip install --upgrade --force-reinstall sentence-transformers

#from langchain.embeddings import HuggingFaceEmbeddings

from transformers import GPT2LMHeadModel, GPT2Tokenizer


with open('api_HF.txt', 'r') as f:
 api_key = f.read().strip()
# Load the LLAMA 2 7B model and tokenizer from the local directory
#model = GPT2LMHeadModel.from_pretrained(model_directory)
#tokenizer = GPT2Tokenizer.from_pretrained(model_directory)

#from sentence_transformers import SentenceTransformer

#model = SentenceTransformer("BAAI/bge-large-en-v1.5")
# Create embeddings using HuggingFaceEmbeddings (adjust as needed)
#embeddings = HuggingFaceEmbeddings(model_name="llama-2-7b-chat.ggmlv3.q4_0")#"BAAI/bge-small-en-v1.5") 
from langchain.embeddings import HuggingFaceEmbeddings

# Instantiate the HuggingFaceEmbeddings class with the LLAMA 2 model name and API key
model_name = "llama-2-7b-chat.ggmlv3.q4_0"  # Replace with the specific LLAMA 2 model name
api_key = "your_api_key_here"  # Replace with your actual API key

# Create embeddings using HuggingFaceEmbeddings
model_directory = "Model"  
import sentence_transformers
print(sentence_transformers.__file__)


RuntimeError: Failed to import transformers.integrations.integration_utils because of the following error (look up to see its traceback):
Failed to import transformers.modeling_tf_utils because of the following error (look up to see its traceback):
DLL load failed while importing _errors: The specified procedure could not be found.

In [6]:
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import LlamaForCausalLM, LlamaTokenizer
model_path = "Model2"  # Replace with the actual path to your model folder


In [9]:

from langchain.embeddings import HuggingFaceEmbeddings
from transformers import AutoTokenizer, AutoModel

model_name = "Model3"  # Replace with your chosen model
model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model=model,
    tokenizer=tokenizer
)

ValidationError: 2 validation errors for HuggingFaceEmbeddings
model
  extra fields not permitted (type=value_error.extra)
tokenizer
  extra fields not permitted (type=value_error.extra)

In [ ]:
embeddings = HuggingFaceEmbeddings(
    model_name="Model",
    model_kwargs={'temperature': 0.7, 'max_new_tokens': 256}
)




RuntimeError: Failed to import transformers.integrations.integration_utils because of the following error (look up to see its traceback):
Failed to import transformers.modeling_tf_utils because of the following error (look up to see its traceback):
DLL load failed while importing _errors: The specified procedure could not be found.

In [ ]:
# Use the embeddings object to encode input text and obtain embeddings
input_text = "Your input text here"
#encoded_text = embeddings.encode(input_text)

In [ ]:
#embeddings = OpenAIEmbeddings(api_key=api_key)

c:\Anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [ ]:
from langchain.vectorstores import FAISS

In [ ]:
vector_store = FAISS.from_documents(document_answer_gen, embeddings)

NameError: name 'embeddings' is not defined

In [ ]:
llm_answer_gen = ChatOpenAI(temperature=0.1, model="gpt-3.5-turbo")

In [ ]:
ques

'1. What is the main goal of the Sustainable Development Goals (SDGs) mentioned in the text?\n2. How many countries are involved in the Sustainable Development Goals initiative?\n3. What progress has been made in reducing extreme poverty in the past 15 years?\n4. How has hunger been addressed in the past 20 years and what further steps are proposed?\n5. What are some of the key components of the goal to ensure healthy lives and promote well-being for all at all ages?\n6. What progress has been made in achieving universal primary education and what are the next steps?\n7. How have gender inequalities been addressed in terms of education and employment?\n8. What is the goal related to access to clean drinking water and sanitation by 2030?\n9. How has access to electricity improved globally and what are the challenges associated with energy consumption?\n10. How can technological progress and innovation contribute to sustainable economic growth and job creation?'

In [ ]:
ques_list = ques.split("\n")

In [ ]:
ques_list

['1. What is the main goal of the Sustainable Development Goals (SDGs) mentioned in the text?',
 '2. How many countries are involved in the Sustainable Development Goals initiative?',
 '3. What progress has been made in reducing extreme poverty in the past 15 years?',
 '4. How has hunger been addressed in the past 20 years and what further steps are proposed?',
 '5. What are some of the key components of the goal to ensure healthy lives and promote well-being for all at all ages?',
 '6. What progress has been made in achieving universal primary education and what are the next steps?',
 '7. How have gender inequalities been addressed in terms of education and employment?',
 '8. What is the goal related to access to clean drinking water and sanitation by 2030?',
 '9. How has access to electricity improved globally and what are the challenges associated with energy consumption?',
 '10. How can technological progress and innovation contribute to sustainable economic growth and job creation

In [ ]:
from langchain.chains import RetrievalQA

In [ ]:
answer_generation_chain = RetrievalQA.from_chain_type(llm=llm_answer_gen, 
                                               chain_type="stuff", 
                                               retriever=vector_store.as_retriever())


In [ ]:
# Answer each question and save to a file
for question in ques_list:
    print("Question: ", question)
    answer = answer_generation_chain.run(question)
    print("Answer: ", answer)
    print("--------------------------------------------------\\n\\n")
    # Save answer to file
    with open("answers.txt", "a") as f:
        f.write("Question: " + question + "\\n")
        f.write("Answer: " + answer + "\\n")
        f.write("--------------------------------------------------\\n\\n")





Question:  1. What is the main goal of the Sustainable Development Goals (SDGs) mentioned in the text?
Answer:  The main goal of the Sustainable Development Goals (SDGs) mentioned in the text is to end extreme poverty in all forms by 2030.
--------------------------------------------------\n\n
Question:  2. How many countries are involved in the Sustainable Development Goals initiative?
Answer:  193 countries are involved in the Sustainable Development Goals initiative.
--------------------------------------------------\n\n
Question:  3. What progress has been made in reducing extreme poverty in the past 15 years?
Answer:  In the past 15 years, the international community has successfully cut extreme poverty in half.
--------------------------------------------------\n\n
Question:  4. How has hunger been addressed in the past 20 years and what further steps are proposed?
Answer:  In the past 20 years, hunger has been addressed by reducing it by almost half. Many countries that used to 